LLaMA 3 8B Instruct (Hugging Face, format HF Transformers .bin/.safetensors)
→ modèle PyTorch
→ export ONNX
→ optimisation NPU Qualcomm


In [1]:
!pip install transformers==4.40.1
!pip install torch onnx onnxruntime



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 38.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.0.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 22.5 MB/s eta 0:00:0000:0100:01


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)


## Export du modèle:

Chargement via HuggingFace du modèle et export du modèle vers format onnx

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)

dummy_input = tokenizer("Explique la Révolution française", return_tensors="pt")
input_ids = dummy_input["input_ids"]

# Important: cast to float32 or float16 selon compatibilité AI Hub
input_ids = input_ids.to(torch.int32)

torch.onnx.export(
    model,
    (input_ids,),
    "llama3-8b-instruct.onnx",
    input_names=["input_ids"],
    output_names=["logits"],
    opset_version=17,
    do_constant_folding=True,
    dynamic_axes={"input_ids": {0: "batch", 1: "seq"}}
)


---

## Test du modèle convertis vers format dlc

In [ ]:
python -m onnxruntime.tools.symbolic_shape_infer llama3-8b-instruct.onnx

---

## Conversion onnx -> dlc pour inférence via NPU

dlc est le format compatible pour inférer avec le NPU

In [ ]:
snpe-onnx-to-dlc --input_network llama3-8b-instruct.onnx --output_path llama3-8b-instruct.dlc --allow_unconsumed_nodes --keep_unsigned --enable_batchnorm_fold
